# Procesar los datos de los videos

Este notebook se encarga de transformar los datos crudos provenientes de archivos de video en un formato estructurado y útil para el análisis. Parte de archivos que almacenan la posicion tridimensional de distintos puntos del cuerpo, después calcula el ángulo entre disintos puntos del cuepro, y por último se realizan cálculos estadísticos sobre los ángulos

**Objetivo**
*  Leer y procesar los archivos crudos para crear un DataFrame que contenga información esencial sobre cada grabación, incluyendo detalles sobre el paciente, el número de repetición, el ID del movimiento, la precisión del gesto (si es correcto o no), y la posición de los keypoints.
* Crear un segundo DataFrame que contenga los ángulos calculados entre algunos de los keypoints, lo que ayudará a entender mejor la postura y el movimiento durante la ejecución del gesto.
* Sobre este último, realizar cálculos estadísticos sobre los ángulos calculados entre keypoints para cada repetición y gesto. 

**Entrada**
* Archivos de datos de videos dentro del directorio ``SkeletonData/RawData``, que contienen información sobre el paciente, las repeticiones, los IDs de movimiento, y las posiciones de keypoints.

**Salida**
* ``Resultados/raw_pacientes.csv``: contiene información detallada sobre cada grabación. Es el archivo de entrada de ``analisis_datos.ipynb``.
* ``Resulados/angles.csv``: incluye los ángulos calculados entre keypoints, facilitando el análisis de la postura y el movimiento durante el gesto. Este será el archivo de entrada de ``medidas_angulos.ipynb`` y de ``analisis_datos.ipynb``.
* ``Resultados/medidasPerRepetition.csv``: contiene una fila por repetición y gesto, que incluye estadísticas para cada ángulo calculado.

**Índice**
1. [Procesar Raw Data Folder](#1-procesar-raw-data-folder)\
    1.1 [Datos de los nombres de los archivos](#11-datos-de-los-nombres-de-los-archivos)\
    1.2 [Datos de los archivos](#12-datos-de-los-archivos)
2. [Calcular los ángulos](#2-calcular-los-ángulos)
3. [Cálculos estadísticos sobre los ángulos](#3-cálculos-estadísticos-sobre-los-ángulos)


In [1]:
# importar librerias necesarias
import pandas as pd # para manejar dataframes
import numpy as np
import functions as fn
import os

# Para ignorar los FutureWarning
import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)

*****
## 1. Procesar Raw Data Folder

El conjunto de datos incluye a 30 sujetos, de los cuales 14 son sanos y 16 son pacientes. Cada participante realiza 9 tipos de ejercicios, con 6 repeticiones de cada uno.

La carpeta contiene 2598 archivos de texto con información de los videos, donde cada archivo corresponde a un sujeto realizando una repetición de un gesto. Hay información tanto en el contenido del archivo como en el propio nombre del archivo.

* En el nombre del archivo se encuentran los identificadores del sujeto, del día, del gesto, el número de repetición, si es correcto y la posición del sujeto durante el ejercicio.

* Dentro de cada archivo hay información sobre las coordenadas de 25 puntos del cuerpo captados por un sensor que graba a 30 fps. Por lo tanto, por cada segundo de video, se generan 30 líneas en el archivo de texto. Cada línea sigue el siguiente formato: marca de tiempo, XX, XX, seguido de 25 pares de (NombreArticulación, EstadoSeguimiento, coordenada 3D X, coordenada 3D Y, coordenada 3D Z, coordenada 2D X, coordenada 2D Y).


### 1.1 Datos de los nombres de los archivos

Los nombre de los archivos siguen el siguiente formato: *SubjectID_DateID_GestureLabel_RepetitionNumber_CorrectLabel_Position.txt*

* *SubjectID*: id uniquely identifying the person performing the exercise
* *DateID*: id identifying the session in which the person was recorded
* *GestureLabel*: Label identifying the gesture; possible values are from 0 to 8
* *RepetitionNumber*: Each gesture was repeated several times and this shows the repetition number
* *CorrectLabel*: A value of 1 represents a gesture labeled as being correctly executed, while a value of 2 is for a gesture labeled as incorrect
* *Position*: Some of the persons performed the gestures sitting on a chair or wheelchair, while others standing

En el archivo `functions.py` se ha creado la siguiente función:
<small>
```python
def leer_nombre_archivo(archivo:str) -> list[str]:
    """
    Extrae datos específicos del nombre de un archivo de texto.

    Parámetros
    ----------
    archivo : str
        Nombre del archivo, con el formato
        'SubjectID_DateID_GestureLabel_RepetitionNumber_CorrectLabel_Position.txt'.

    Return
    -------
    campos : list[str]
        Lista de cadenas de texto que contiene los datos extraídos del nombre del archivo:
        [SubjectID, DateID, GestureLabel, RepetitionNumber, CorrectLabel, Position].
    """
    archivo = archivo.split('.')[0] # quita la extension txt
    campos = archivo.split('_') # separa los campos por _
    return campos
```
</small>


### 1.2 Datos de los archivos
Each raw data file contains per line: timestamp, XX, XX, followed by a 25 pairs of (JointName, TrackedStatus, 3d coordinate X, 3d coordinate Y, 3d coordinate Z, 2d coordinate X, 2d coordinate Y)

In [2]:
# Función para extraer la información de dentro de los archivos
def leer_datos_archivo(directorio:str, columnas:list[str]) -> pd.DataFrame:
    """
    Compila la información de los archivos en un directorio y los guarda en un DataFrame.

    Parámetros
    ----------
    directorio : str
        Nombre del directorio donde se encuentran los archivos.
    columnas : list[str]
        Lista con los nombres de las columnas para el DataFrame de salida.

    Return
    -------
    pd.DataFrame
        DataFrame con todos los datos recopilados de los archivos.
    """
    # Crea una lista con los nombres de los archivos en el directorio
    file_list = os.listdir(directorio)

    # lista para almacenar los datos extraídos
    list_data = []

    # Itera sobre cada archivo
    for file_name in file_list:
         # Extrae los datos del nombre del archivo
        campos = fn.leer_nombre_archivo(file_name)

        with open(os.path.join(directorio, file_name), 'r') as file:
            for line in file:
                 # Divide la línea por comas y extrae la información deseada
                 # omitiendo timestamp y otros datos innecesarios
                line_data = line.strip().split(',')[3:]
                # Quita los paréntesis
                cleaned_data = [item.replace('(', '').replace(')', '') for item in line_data] 
                # por cada linea de los archivos necesitamos bloques de 7 valores
                for i in range(0, len(cleaned_data), 7): 
                    list_data.append(campos + cleaned_data[i:i + 7])
    df = pd.DataFrame(list_data, columns=columnas)
    return df

In [3]:
# directorio donde se encuentran los datos
directory = '../dataset/SkeletonData/RawData' 

columnas = ['SubjectID', 'DateID', 'GestureLabel', 'RepetitionNumber', 'CorrectLabel', 'Position',
            'JointName', 'TrackedStatus', '3D_X', '3D_Y', '3D_Z', '2D_X', '2D_Y']

# Extraer la informacion y almacenarla en un DataFrame
df_data = leer_datos_archivo(directory, columnas)

In [4]:
# Eliminar columnas innceserarias
df_data.drop(['TrackedStatus', 'DateID', 'Position', '2D_X', '2D_Y'], axis=1, inplace=True)

# Añadir columna más descriptiva de los gestos
gesture_mapping = {
    '0': 'EFL',
    '1': 'EFR',
    '2': 'SFL',
    '3': 'SFR',
    '4': 'SAL',
    '5': 'SAR',
    '6': 'SFE',
    '7': 'STL',
    '8': 'STR'
}

# Nueva columna basada en el mapeo
columna_gestrure_name = df_data['GestureLabel'].map(gesture_mapping)

# Insertar la columna al lado de GestureLabel
df_data.insert(2, 'GestureName', columna_gestrure_name)

df_data

,SubjectID,GestureLabel,GestureName,RepetitionNumber,CorrectLabel,JointName,3D_X,3D_Y,3D_Z
0,102,0,EFL,10,1,SpineBase,-0.08088344,-0.2248836,2.661578
1,102,0,EFL,10,1,SpineMid,-0.06806522,0.04492111,2.628779
2,102,0,EFL,10,1,Neck,-0.055614,0.3082771,2.583972
3,102,0,EFL,10,1,Head,-0.04478608,0.4328104,2.593495
4,102,0,EFL,10,1,ShoulderLeft,-0.2232155,0.2021449,2.549825
...,...,...,...,...,...,...,...,...,...
5500495,307,8,STR,9,1,SpineShoulder,-0.05799517,0.5291457,2.422904
5500496,307,8,STR,9,1,HandTipLeft,-0.302538,-0.1131345,2.284269
5500497,307,8,STR,9,1,ThumbLeft,-0.2783904,-0.06298634,2.269769
5500498,307,8,STR,9,1,HandTipRight,0.1525867,-0.136378,2.45287


In [5]:
# guardar los datos en formato csv
df_data.to_csv('../Resultados/raw_pacientes.csv', index=False)

******
## 2. Calcular los ángulos

<div style="text-align: center;">
<img src="../Imagenes/gestures.png" width="500"/>
</div>

En el archivo `functions.py` se ha creado la siguiente función:
<small>
```python
def apply_angles_I(df: pd.DataFrame) -> pd.DataFrame:
    '''Aplica la función para caluclar los ángulos a los datos en crudo
    para la fase 1''' 
    angles = []

    # Agrupar el DataFrame por cada 25 filas e iterar
    for _, group in df.groupby(np.arange(len(df)) // 25):
        # Calculo de angulos
        left_arm_angle = calculate_angle(group, 'ShoulderLeft', 'ElbowLeft',        'WristLeft')
        left_armpit_angle = calculate_angle(group, 'HipLeft', 'ShoulderLeft', 'ElbowLeft')
        left_wrist_angle = calculate_angle(group, 'ElbowLeft', 'WristLeft', 'HandLeft')
        left_wrist_vertical_angle = calculate_angle(group, 'SpineBase', 'SpineShoulder', 'WristLeft')
        left_elbow_vertical_angle = calculate_angle(group, 'SpineBase', 'SpineShoulder', 'ElbowLeft')
        left_shoulder_angle = calculate_angle(group, 'SpineShoulder', 'ShoulderLeft', 'ElbowLeft')
        right_arm_angle = calculate_angle(group, 'ShoulderRight', 'ElbowRight', 'WristRight')
        right_armpit_angle = calculate_angle(group, 'HipRight', 'ShoulderRight', 'ElbowRight')
        right_wrist_angle = calculate_angle(group, 'ElbowRight', 'WristRight', 'HandRight')
        right_wrist_vertical_angle = calculate_angle(group, 'SpineBase', 'SpineShoulder', 'WristRight')
        right_elbow_vertical_angle = calculate_angle(group, 'SpineBase', 'SpineShoulder', 'ElbowRight')
        right_shoulder_angle = calculate_angle(group, 'SpineShoulder', 'ShoulderRight', 'ElbowRight')
        hip_angle_left = calculate_angle(group, 'HipLeft', 'SpineBase', 'KneeLeft')
        knee_angle_left = calculate_angle(group, 'HipLeft', 'KneeLeft', 'AnkleLeft')
        ankle_angle_left = calculate_angle(group, 'KneeLeft', 'AnkleLeft', 'FootLeft')
        hip_angle_right = calculate_angle(group, 'HipRight', 'SpineBase', 'KneeRight')
        knee_angle_right = calculate_angle(group, 'HipRight', 'KneeRight', 'AnkleRight')
        ankle_angle_right = calculate_angle(group, 'KneeRight', 'AnkleRight', 'FootRight')

        # Extraer columnas adicionales
        additional_data = group.iloc[0][['SubjectID', 'GestureLabel', 'GestureName', 'RepetitionNumber', 'CorrectLabel']]

        # Almacenar la información en un diccionario
        angles.append({
            **additional_data,
            'LeftArmAngle': left_arm_angle,
            'LeftArmpitAngle': left_armpit_angle,
            'LeftWristAngle': left_wrist_angle,
            'LeftWristVerticalAngle': left_wrist_vertical_angle,
            'LeftElbowVerticalAngle': left_elbow_vertical_angle,
            'LeftShoulderAngle': left_shoulder_angle,
            'RightArmAngle': right_arm_angle,
            'RightArmpitAngle': right_armpit_angle,
            'RightWristAngle': right_wrist_angle,
            'RightWristVerticalAngle': right_wrist_vertical_angle,
            'RightElbowVerticalAngle': right_elbow_vertical_angle,
            'RightShoulderAngle': right_shoulder_angle,
            'HipAngleLeft': hip_angle_left,
            'KneeAngleLeft': knee_angle_left,
            'AnkleAngleLeft': ankle_angle_left,
            'HipAngleRight': hip_angle_right,
            'KneeAngleRight': knee_angle_right,
            'AnkleAngleRight': ankle_angle_right
        })

    # Crear un DataFrame a partir de la lista de diccionarios
    return pd.DataFrame(angles)
```
</small>

In [6]:
# Aplica la función para caluclar los ángulos a los datos en crudo
df_angles = fn.apply_angles_I(df_data)
# Visualizar el dataframe
df_angles

,SubjectID,GestureLabel,GestureName,RepetitionNumber,CorrectLabel,LeftArmAngle,LeftArmpitAngle,LeftWristAngle,LeftWristVerticalAngle,LeftElbowVerticalAngle,...,RightWristAngle,RightWristVerticalAngle,RightElbowVerticalAngle,RightShoulderAngle,HipAngleLeft,KneeAngleLeft,AnkleAngleLeft,HipAngleRight,KneeAngleRight,AnkleAngleRight
0,102,0,EFL,10,1,36.445861,149.337083,4.831462,142.905759,134.653133,...,13.634312,139.166534,131.770369,55.194640,132.747369,99.561246,79.739535,143.350817,105.108123,81.345980
1,102,0,EFL,10,1,36.501008,149.402103,16.758538,143.001279,134.716086,...,1.784125,139.092182,131.718517,55.180202,132.648786,99.678021,80.376922,143.683790,105.217669,81.391368
2,102,0,EFL,10,1,36.431684,149.368710,17.130884,142.965117,134.688924,...,7.832669,138.919339,131.706943,55.151514,132.684043,99.996248,82.609605,143.639654,105.216908,81.310906
3,102,0,EFL,10,1,36.645339,149.327745,6.310039,142.880512,134.663061,...,30.207149,122.588844,130.809573,54.387967,132.864128,99.998782,82.691058,142.434449,102.783610,82.313890
4,102,0,EFL,10,1,36.704133,149.420443,5.198054,142.876385,134.672606,...,48.424470,120.807916,126.042393,48.220668,132.908134,99.745980,80.982213,142.287131,102.197980,82.432998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220015,307,8,STR,9,1,19.235168,156.103380,10.597409,149.864559,142.621011,...,8.590306,159.090476,146.675015,62.828171,107.710008,17.152761,57.808079,94.948037,1.709497,41.616584
220016,307,8,STR,9,1,19.337504,156.056261,9.097187,149.734213,142.577680,...,8.083043,159.185684,146.713406,62.858293,107.352914,16.479229,57.733793,94.653231,1.263963,41.879291
220017,307,8,STR,9,1,19.762179,156.130164,9.288112,149.712017,142.588822,...,10.157666,159.256181,146.753525,63.024077,107.398951,16.368690,58.670153,94.346427,1.041109,78.704138
220018,307,8,STR,9,1,20.339779,156.114139,7.335038,149.496620,142.536620,...,9.108455,159.332153,146.802282,63.054702,107.436579,16.163032,57.798046,94.144821,1.015447,78.752737


In [7]:
# guardar los datos en formato csv
df_angles.to_csv('../Resultados/angles.csv', index=False)

******
## 3. Cálculos estadísticos sobre los ángulos

Realizar cálculos estadísticos sobre los ángulos calculados entre keypoints para cada repetición y gesto. 

In [8]:
# Cálculos estadísticos sobre los ángulos
def calculos_estadísticos(df:pd.DataFrame) -> pd.DataFrame:
    """
    Realiza cálculos estadísticos sobre los ángulos en un
    DataFrame agrupado por sujeto, gesto y número de repetición.

    Parámetros
    ----------
    df : pd.DataFrame
        DataFrame que contiene información sobre los ángulos,
        así como otras columnas relacionadas con el sujeto,
        gesto, repetición, etc.

    Returns
    -------
    pd.DataFrame
        DataFrame que contiene las estadísticas descriptivas
        calculadas para cada grupo de ángulos, con una fila por
        combinación de sujeto, gesto y repetición.
    """

    # Agrupa el DataFrame 
    groups = df.groupby(["SubjectID", "GestureLabel", "RepetitionNumber"])

    # Lista para almacenar los datos de salida
    data = []

    # Itera sobre cada grupo
    for (subject_id, gesture_label, repetition_number), group in groups:
        # Selecciona solo las columnas que contienen los ángulos 
        angles = group.iloc[:, 6:]

        # Calcula la media y la desviación estándar para los ángulos
        means = angles.mean()
        std_devs = angles.std()

        # Almacena las estadísticas en un diccionario 
        data.append({
            'SubjectID': subject_id,
            'GestureLabel': gesture_label,
            'GestureName': group['GestureName'].iloc[0],
            'RepetitionNumber': repetition_number,
            'CorrectLabel': group['CorrectLabel'].iloc[0],
            'Duration': len(group),  # Duración en número de frames
            'standardDeviation': std_devs,
            'Maximum': angles.max(),
            'Minimum': angles.min(),
            'Mean': means,
            'Range': angles.max() - angles.min(),
            'Variance': angles.var(),
            'CoV': std_devs / means,  # Coeficiente de variación
            'Skewness': angles.skew(),  # Asimetría
            'Kurtosis': angles.kurtosis()  # Curtosis
        })

    # Convierte la lista de diccionarios en un DataFrame y lo ordena
    df_stats = pd.DataFrame(data)
    df_stats = df_stats.sort_values(['SubjectID', 'GestureLabel', 'RepetitionNumber'])

    return df_stats

In [9]:
# Cálculos sobre los ángulos
df_stats = calculos_estadísticos(df_angles)
df_stats.head()

,SubjectID,GestureLabel,GestureName,RepetitionNumber,CorrectLabel,Duration,standardDeviation,Maximum,Minimum,Mean,Range,Variance,CoV,Skewness,Kurtosis
0,102,0,EFL,1,1,74,LeftArmpitAngle 0.330174 LeftWrist...,LeftArmpitAngle 150.523771 LeftWris...,LeftArmpitAngle 149.199484 LeftWris...,LeftArmpitAngle 149.796718 LeftWris...,LeftArmpitAngle 1.324287 LeftWris...,LeftArmpitAngle 0.109015 LeftWri...,LeftArmpitAngle 0.002204 LeftWristA...,LeftArmpitAngle 0.149918 LeftWristA...,LeftArmpitAngle -0.988983 LeftWrist...
1,102,0,EFL,10,1,30,LeftArmpitAngle 0.190665 LeftWrist...,LeftArmpitAngle 149.521752 LeftWris...,LeftArmpitAngle 148.815587 LeftWris...,LeftArmpitAngle 149.163310 LeftWris...,LeftArmpitAngle 0.706164 LeftWris...,LeftArmpitAngle 0.036353 LeftWri...,LeftArmpitAngle 0.001278 LeftWristA...,LeftArmpitAngle -0.202535 LeftWristA...,LeftArmpitAngle -0.735289 LeftWristA...
2,102,0,EFL,11,1,46,LeftArmpitAngle 0.322199 LeftWrist...,LeftArmpitAngle 149.937644 LeftWris...,LeftArmpitAngle 148.972360 LeftWris...,LeftArmpitAngle 149.473337 LeftWris...,LeftArmpitAngle 0.965284 LeftWris...,LeftArmpitAngle 0.103812 LeftWri...,LeftArmpitAngle 0.002156 LeftWristA...,LeftArmpitAngle 0.024644 LeftWristA...,LeftArmpitAngle -1.579874 LeftWristA...
3,102,0,EFL,12,1,49,LeftArmpitAngle 0.345296 LeftWrist...,LeftArmpitAngle 149.855548 LeftWris...,LeftArmpitAngle 148.683028 LeftWris...,LeftArmpitAngle 149.441613 LeftWris...,LeftArmpitAngle 1.172520 LeftWris...,LeftArmpitAngle 0.119229 LeftWri...,LeftArmpitAngle 0.002311 LeftWristA...,LeftArmpitAngle -1.216171 LeftWristA...,LeftArmpitAngle 0.152294 LeftWrist...
4,102,0,EFL,2,1,64,LeftArmpitAngle 0.843230 LeftWrist...,LeftArmpitAngle 151.502280 LeftWris...,LeftArmpitAngle 148.815717 LeftWris...,LeftArmpitAngle 150.152401 LeftWris...,LeftArmpitAngle 2.686563 LeftWris...,LeftArmpitAngle 0.711036 LeftWri...,LeftArmpitAngle 0.005616 LeftWristA...,LeftArmpitAngle 0.063879 LeftWristA...,LeftArmpitAngle -1.293845 LeftWrist...


Como se ve en la cabecera del DataFrame anterior es poco legible, por lo que se va a formatear para mejorar la legibilidad del DataFrame y evitar la representación diccionarios dentro de las columnas del DataFrame

Se he creado una función para formatear las columnas en:

<small>

```python

def formatear_columnas(df:pd.DataFrame, nombre_columna:str) -> pd.DataFrame: 
    '''
    Esta función toma una columna de un DataFrame que contiene diccionarios
    y expande cada diccionario en nuevas columnas, donde cada nueva columna
    corresponde a:  clave del diccionario + _ + nombre de la columna original. 

    Parámetro
    --------
    nombre_columna : str
        Nombre de la columna del DataFrame que contiene los diccionarios.

    Return
    --------
     Un nuevo DataFrame con columnas separadas para cada clave de los diccionarios.
    '''
    keys = set().union(*(d.keys() for d in df[nombre_columna]))
    data = {}
    for key in keys:
        data[key + '_' + nombre_columna] = df[nombre_columna].apply(lambda x: x.get(key))
    return pd.DataFrame(data)

```

</small>

In [10]:
columnas = ['standardDeviation', 'Maximum', 'Minimum', 'Mean', 'Range',
            'Variance', 'CoV', 'Skewness', 'Kurtosis']
# Reformatea cada columna de diccionario y concatena los resultados
nuevas_columnas = pd.concat([fn.formatear_columnas(df_stats[col], col) for col in columnas], axis=1)

# Concatenar las nuevas columnas con el DataFrame original
df = pd.concat([df_stats, nuevas_columnas], axis=1)

# Elimina las columnas originales que contenían diccionarios
df = df.drop(columnas, axis=1)

# Ordena el DataFrame 
## pasamos a numerico para que ordene bien los numeros
df = df.apply(pd.to_numeric, errors='ignore')
df = df.sort_values(['SubjectID', 'GestureLabel', 'RepetitionNumber'])

# Mostrar el resultado
df

,SubjectID,GestureLabel,GestureName,RepetitionNumber,CorrectLabel,Duration,LeftShoulderAngle_standardDeviation,RightElbowVerticalAngle_standardDeviation,AnkleAngleRight_standardDeviation,RightArmpitAngle_standardDeviation,...,RightWristAngle_Kurtosis,RightWristVerticalAngle_Kurtosis,LeftArmpitAngle_Kurtosis,HipAngleLeft_Kurtosis,RightShoulderAngle_Kurtosis,KneeAngleLeft_Kurtosis,AnkleAngleLeft_Kurtosis,HipAngleRight_Kurtosis,KneeAngleRight_Kurtosis,LeftElbowVerticalAngle_Kurtosis
0,102,0,EFL,1,1,74,1.084519,3.096725,1.735763,4.022542,...,0.375607,-1.476049,-0.988983,0.195086,2.216565,-0.166965,-0.921846,-1.213044,-0.159311,-0.555001
4,102,0,EFL,2,1,64,0.643176,2.853585,2.519931,5.171543,...,1.154400,-1.555020,-1.293845,-0.788811,0.336867,-1.055132,-1.415922,0.166379,-0.795236,-1.487894
5,102,0,EFL,3,1,58,0.872738,3.884793,0.899799,6.454416,...,9.590802,-1.550443,-1.244715,-0.915824,1.852582,-1.142128,-1.203117,-1.304679,-0.629462,-1.585167
6,102,0,EFL,4,1,60,0.779144,3.158293,1.140919,5.178969,...,4.133624,-1.556884,-1.533188,-1.185226,1.037615,-1.222738,-1.515334,0.005324,-0.862310,-1.624883
7,102,0,EFL,5,1,63,0.752460,2.965702,1.915376,5.606324,...,5.640903,-1.417592,-1.258867,-0.592432,1.301458,0.010318,-0.517289,-0.484635,-0.721140,-1.522244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2480,307,8,STR,8,1,48,1.000586,1.546787,15.325924,1.555034,...,0.056064,-1.377709,-0.781945,-1.222994,0.297255,-0.347162,0.312861,-1.014746,-0.038718,-0.714678
2481,307,8,STR,9,1,48,0.628521,1.306536,17.736973,1.332626,...,-1.006323,-1.780586,-1.644095,-1.672822,-1.030659,-0.874793,1.056307,-1.651462,-1.195026,-1.786392
2471,307,8,STR,10,1,47,0.776700,1.271030,18.029198,1.315829,...,-0.776532,-1.360834,-0.826005,-1.403915,-1.114165,-1.055977,-0.851119,-1.103517,-0.471100,-0.839798
2472,307,8,STR,11,1,51,1.054304,1.844722,15.416612,1.896625,...,-0.052683,-1.761455,-0.620023,-1.462976,-0.647248,-1.100134,-0.156794,-1.697380,-0.274878,-0.745076


In [11]:
# Guardar el DataFrame en formato csv
df.to_csv('../Resultados/medidasPerRepetition.csv', index=False)